In [1]:
from datetime import datetime
import requests

response = requests.get('http://atlas.obs-hp.fr/sophie/sophie.cgi?n=sophies&a=htab&ob=ra,seq&c=o&o=51%20peg')

print(response.status_code)

200


In [2]:
import bs4

# Obtener el html

soup = bs4.BeautifulSoup(response.text, 'html.parser')

xpath_title = '//table[@class="datatable"]/tbody/tr/td[7]/text()'

query = soup.select('table[class="datatable"] tbody tr')

print(len(query))
#print(query[1].select('td')[6])
dates = []
for i in range(0, len(query)):
    #print(query[i].select('td')[6].text)
    dates.append(query[i].select('td')[6].text)
print('dates: ', len(dates))

183
dates:  183


In [3]:
import bs4

# Obtener el html

soup = bs4.BeautifulSoup(response.text, 'html.parser')

xpath_title = '//table[@class="datatable"]/thead/tr/th/a/text()'


query = soup.select('table[class="datatable"] thead th')


# Head

In [4]:
#query = soup.select('table[class="datatable"] thead tr th a')
#header = [querys.text for querys in query]


def _get_headers():
    row_header = soup.select('table[class="datatable"] thead tr th a')
    # table_body's header
    headers = [header.text for header in row_header]

    # List of data I want to get
    data_list = ['date', 'fiber_b', 'sn26', 'view_head', 'search_ccf']  
    position = []
    for i in range(0, len(headers)):
        for data in data_list:
            if headers[i] == data:
                position.append(i)

    columns = {}
    for i in range(0, len(data_list)):
        columns[data_list[i]] = position[i]
    return headers, columns
_get_headers()

(['objname',
  'RA (J2000) Dec',
  'S',
  'E',
  'seq',
  'slen',
  'date',
  'mode',
  'fiber_b',
  'exptime',
  'sn26',
  'view_spec',
  'view_head',
  'get_spec',
  'get_e2ds',
  'customize',
  'search_ccf'],
 {'date': 6, 'fiber_b': 8, 'sn26': 10, 'view_head': 12, 'search_ccf': 16})

In [5]:

query = soup.select('table[class="datatable"] thead th')
print(len(query))

header = []
for i in range(0,len(query)):
    header.append(query[i].text)

lista = ['date', 'fiber_b', 'sn26', 'view_spec', 'search_ccf']    
posicion = []
for i in range(0,len(header)):
    for j in range(0,len(lista)):
        if header[i] == lista[j]:
            posicion.append(i)

diccionario ={}
for i in range(0, len(lista)):
    diccionario[lista[i]]=posicion[i]
diccionario

17


{'date': 6, 'fiber_b': 8, 'sn26': 10, 'view_spec': 11, 'search_ccf': 16}

# Body

In [6]:
query = soup.select('table[class="datatable"] tbody tr')


print(query[3].select('td')[16].select('a')[0])
#print(query[0])

<a href="sophie.cgi?n=sophiecc&amp;a=hexp&amp;fql=[seq =825680]">search_CCF</a>


In [7]:
#Codigo de ejemplo platzi
def article_links(self):
    link_list = []
    for link in self._select(self._queries['homepage_article_links']):
        if link and link.has_attr('href'):
            link_list.append(link)

    return set(link['href'] for link in link_list)

In [8]:
def column():    
    query = soup.select('table[class="datatable"] tbody tr')
    
    select_a = []
    for i in range(0, len(query)):
        data = query[i].select('td')[12].select('a')
        
        select_a.append(data)
    
    link_list = []
    none_url = []
    for index, link in enumerate(select_a):
        if len(link) == 0:
            none_url.append(index)
        elif link[0] and link[0].has_attr('href'):
            link_list.append(link)

            
    return list(link[0]['href'] for link in link_list), none_url


In [9]:
a, none_url_list = column()
print(a)
print(none_url_list)

['sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,671480]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,815760]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,815780]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,825680]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,847890]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,848960]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1014070]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1203340]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1217920]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1225370]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1244250]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,133040]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,133820]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,137810]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,139920]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,145770]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,146670]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,147590]', 'sop

# Borrar datos faltantes

In [10]:
def _obtein_data(rows, column_name, none_url_list):

    results = []
    for i in range(0, len(rows)):
        data = rows[i].select('td')[column_name].text
        if len(data) !=0:
            results.append(data)
        elif len(data) == 0:
            results.append('')
    
    results_clean = np.delete(results, none_url_list).tolist()
    return results_clean

In [11]:
import numpy as np
lista_nombres = ['Ricky', 'Alvaro', 'David', 'Jacinto', 'Jose', 'Ricky', 'Jose', 'Jose']
borrar = [0,1,2]
np.delete(lista_nombres, borrar).tolist()
# Returns: ['l', '42', 'u']

['Jacinto', 'Jose', 'Ricky', 'Jose', 'Jose']

# Corregir link

In [12]:
links, none_url_list = column()
print(links)
print(none_url_list)

['sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,671480]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,815760]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,815780]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,825680]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,847890]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,848960]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1014070]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1203340]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1217920]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1225370]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1244250]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,133040]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,133820]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,137810]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,139920]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,145770]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,146670]', 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,147590]', 'sop

In [13]:
unir_espacios = [link.replace(' ', '') for link in links]
unir_espacios

['sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,671480]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,815760]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,815780]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,825680]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,847890]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,848960]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1014070]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1203340]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1217920]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1225370]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,1244250]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,133040]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,133820]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,137810]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,139920]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,145770]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,146670]',
 'sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2

### http://atlas.obs-hp.fr/sophie/sophie.cgi?n=sophiecc&a=hexp&fql=[seq%20=671480]
### http://atlas.obs-hp.fr/sophie/sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,671480]

In [15]:
def _build_link(links):
    unir_espacios = [link.replace(' ', '') for link in links]
    link_header = 'http://atlas.obs-hp.fr/sophie/'
    
    new_link = []
    for i in unir_espacios:
        new_link.append('{}{}'.format(link_header, i))
    return new_link

In [16]:
print(_build_link(links)[0])

http://atlas.obs-hp.fr/sophie/sophie.cgi?n=sophie&c=i&z=fd&o=sophie:[e2ds,671480]


In [17]:
# cadena.endswith('da') TRUE

# JD. Unir a transform.py la 3ra celda

In [85]:
response = requests.get(_build_link(links)[0])

soup = bs4.BeautifulSoup(response.text, 'html.parser')

In [99]:
query = soup.select('div[class="cizfd"] pre')
x = query[0].text
x = ''
print(x.index('BJD     24'))

ValueError: substring not found

In [95]:

inicio = query[0].text.index('BJD     24')
final = query[0].text.index('Barycentric Julian Day')
JD = float(query[0].text[inicio:final].replace('BJD','').replace(' ', ''))
JD, inicio, final

(2454700.48409259, 38210, 38235)

In [21]:
mensaje9 = "Hola Mundo"
inicio = mensaje9.find('Ho')
final = mensaje9.find('do')
mensaje9[inicio:final]

'Hola Mun'

# Radial Velocity. Unir a transform.py la 3ra celda

In [77]:
response = requests.get('http://atlas.obs-hp.fr/sophie/sophie.cgi?n=sophiecc&a=hexp&fql=[seq%20=671480]')

soup = bs4.BeautifulSoup(response.text, 'html.parser')

In [78]:
print(soup)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html>
<head>
<title>Cross-correlation functions</title> <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="pleinpot.css" rel="stylesheet" type="text/css"/>
<base target="_top"/>
<script src="pleinpot.js"></script>
<script>var pleinpot_version='8.18.10'; var table_description='Cross-correlation functions';var table_name='sophiecc'</script>
</head>
<body>
<table border="0"><tr><td><a href="http://www.obs-hp.fr/"><img alt="Observatoire de Haute Provence (OHP)" border="0" src="logo-OHP.gif"/></a></td><td align="center" width="100%"><font face="helvetica,arial"><h1>The SOPHIE archive</h1></font></td> <td><a href="http://www.obs-hp.fr/guide/sophie/sophie-eng.shtml"><img alt="SOPHIE" border="0" src="sophie_sm.jpg" width="200"/></a></td></tr></table>
<br/><center><h2>
<script>document.write(table_description);</script>
</h2></center><br/>
<div class="cozd">
<di

In [79]:
query = soup.select('table[class="datatable"] tr')
RV = query[7].select('td')[1].text
print(RV)

-33.139 ± 0.0002


In [81]:
class NewsPage:

    def __init__(self, response):
        self.response = response
        
    def radial_velocity(self):
        response = self.response
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        query = soup.select('table[class="datatable"] tr')
        result = query[7].select('td')[1].text
        return result

In [82]:
response = requests.get('http://atlas.obs-hp.fr/sophie/sophie.cgi?n=sophiecc&a=hexp&fql=[seq%20=671480]')
result = NewsPage(response)
print(result.radial_velocity())

-33.139 ± 0.0002


# DataFrame

In [114]:
import pandas as pd
import numpy as np
a= [1,2,4,2,5]
b= [5,3,2,6,7]
c= np.array([a,b])
header = ['asf', 'abc']
df = pd.DataFrame(c.T, columns = header)
print(df)
c.T

,0,1
0,1,5
1,2,3
2,4,2
3,2,6
4,5,7


In [41]:
# python 3.x
import pandas as pd
import numpy as np
df = pd.DataFrame(
    data=np.random.randint(
        0, 10, (6,4)))
df.columns=["a", "b", "c", "d"]
print(df)

   a  b  c  d
0  8  4  4  4
1  4  4  4  6
2  4  4  6  3
3  9  2  9  5
4  3  8  6  6
5  2  5  5  3


In [44]:
df

,asf,abc
0,1,5
1,2,3
2,4,2
3,2,6
4,5,7


In [84]:
print(len(12))

TypeError: object of type 'int' has no len()